# PS2
## MACS 30250, Dr. Evans
## Yiqing Zheng
## Problem 1
(a)

In [94]:
import numpy as np
import pandas as pd
import timeit
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore")

In [95]:
df = pd.read_csv('data\Auto.csv',na_values="?")
df.dropna(inplace=True)
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [96]:
df['orgn1'] = 0
df['orgn1'][df['origin'] == 1] = 1
df['orgn2'] = 0
df['orgn2'][df['origin'] == 2] = 1
df['mpg_high'] = df['mpg']>=df['mpg'].median()
df['mpg_high'].astype('int32')
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,orgn1,orgn2,mpg_high
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,1,0,False
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,1,0,False
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,1,0,False
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,1,0,False
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,1,0,False


In [97]:
y = df['mpg_high'].values
X = df[['cylinders', 'displacement', 'horsepower', 'weight','acceleration', 'year', 'orgn1', 'orgn2']].values

In [103]:
start_time = timeit.default_timer()
mse = []
for i in range(100):
    i += 1
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.35, random_state=i)
    LR = LogisticRegression(n_jobs=1)
    LR.fit(X_train, y_train)
    y_pred = LR.predict(X_test)
    error = ((y_pred != y_test)**2).mean()
    mse.append(error)
    
mse_mean = np.array(mse).mean()
elapsed_time = timeit.default_timer() - start_time
print('Elapsed time=', elapsed_time, 'seconds')
    
print(f"Average error rate = {mse_mean}.")

Elapsed time= 0.3037502000006498 seconds
Average error rate = 0.10152173913043477.


(b)

In [99]:
# First let's check how many cores we have available
import multiprocessing

num_cores = multiprocessing.cpu_count()
print('Number of available cores is', num_cores)

Number of available cores is 4


In [100]:
from dask import compute, delayed
import dask.multiprocessing
from statistics import mean

In [107]:
LR = LogisticRegression(n_jobs=1)
def f(times, seed, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.35, random_state=seed)
    LR.fit(X_train, y_train)
    y_pred = LR.predict(X_test)
    error = ((y_pred != y_test)**2).mean()
    return error

In [113]:
start_time = timeit.default_timer()
lazy_values = []
for i in range(100):
    lazy_values.append(delayed(f)(i+1, i+1, X, y))
    
mse_mean2 = delayed(mean)(lazy_values)

results_par = compute(mse_mean2)

elapsed_time = timeit.default_timer() - start_time
print('Elapsed time=', elapsed_time, 'seconds')
print(f"Average error rate = {results_par[0]}.")

Elapsed time= 0.2131470000022091 seconds
Average error rate = 0.10152173913043479.


The computation time drops while the error rates are the same.